---
## 1. Setup dan Import Libraries

### 📦 Instalasi SimPy

Cell ini akan menginstall library SimPy yang diperlukan untuk menjalankan simulasi. SimPy adalah framework Python untuk discrete-event simulation yang memungkinkan kita untuk:
- Membuat entitas (mobil yang datang)
- Mendefinisikan resource (loket drive-thru)
- Mensimulasikan proses antrian dengan distribusi probabilitas

**Catatan:** Jika menjalankan di Google Colab, library akan otomatis terinstall. Jika sudah terinstall, proses ini akan melewati instalasi.

In [ ]:
# Install SimPy (untuk Google Colab)
!pip install simpy

### 📚 Import Libraries yang Diperlukan

Cell ini mengimport semua library yang akan digunakan dalam simulasi:
- **simpy**: Framework utama untuk discrete-event simulation
- **numpy**: Untuk generate random numbers dari distribusi probabilitas (exponential, uniform)
- **pandas**: Untuk menyimpan dan mengolah data hasil simulasi dalam bentuk DataFrame
- **matplotlib & seaborn**: Untuk membuat visualisasi (histogram, line chart, box plot)

Kita juga set random seed agar hasil simulasi reproducible (dapat diulang dengan hasil yang sama).

In [ ]:
# Import libraries
import simpy
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from collections import defaultdict

# Set style untuk visualisasi
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)

# Set random seed untuk reproducibility
np.random.seed(42)

print("✓ Libraries berhasil di-import")
print(f"SimPy version: {simpy.__version__}")

---
## 2. Konfigurasi Parameter Simulasi

### ⚙️ Definisi Parameter Simulasi

Cell ini mendefinisikan semua parameter yang akan digunakan dalam simulasi sesuai dengan formulir desain:

**Parameter Input:**
- **INTER_ARRIVAL_MEAN = 5**: Rata-rata waktu antar kedatangan mobil (distribusi eksponensial)
- **SERVICE_TIME_MIN/MAX = 3-7**: Range waktu layanan (distribusi uniform)

**Parameter Simulasi:**
- **WARM_UP_PERIOD = 100**: Periode pemanasan untuk menghindari bias kondisi awal kosong
- **SIM_TIME = 1100**: Total waktu simulasi yang dijalankan
- **ANALYSIS_TIME = 1000**: Waktu yang dianalisis (setelah warm-up dibuang)

Dengan parameter ini, kita mensimulasikan sistem drive-thru yang realistis.

In [ ]:
# Parameter Simulasi
INTER_ARRIVAL_MEAN = 5  # Rata-rata waktu antar kedatangan (menit)
SERVICE_TIME_MIN = 3    # Waktu layanan minimum (menit)
SERVICE_TIME_MAX = 7    # Waktu layanan maksimum (menit)
WARM_UP_PERIOD = 100    # Periode warm-up yang akan dibuang (menit)
SIM_TIME = 1100         # Total waktu simulasi (menit)
ANALYSIS_TIME = SIM_TIME - WARM_UP_PERIOD  # Waktu yang dianalisis

print("="*50)
print("PARAMETER SIMULASI DRIVE-THRU")
print("="*50)
print(f"Pola Kedatangan    : Eksponensial (λ = {INTER_ARRIVAL_MEAN} menit)")
print(f"Waktu Layanan      : Uniform ({SERVICE_TIME_MIN}-{SERVICE_TIME_MAX} menit)")
print(f"Warm-up Period     : {WARM_UP_PERIOD} menit")
print(f"Waktu Simulasi     : {SIM_TIME} menit")
print(f"Waktu Analisis     : {ANALYSIS_TIME} menit")
print("="*50)

---
## 3. Implementasi Komponen Simulasi SimPy

### 🏗️ Implementasi Class Simulasi

Cell ini berisi **komponen inti simulasi** dengan class `DriveThruSimulation` yang mengimplementasikan:

**1. Komponen SimPy:**
- **Resource**: Loket drive-thru dengan kapasitas tertentu (1 atau 2 loket)
- **Environment**: Lingkungan simulasi SimPy untuk menjalankan proses

**2. Generator Kedatangan (`car_arrival_generator`):**
- Membangkitkan kedatangan mobil secara kontinyu
- Menggunakan distribusi eksponensial untuk inter-arrival time
- Setiap mobil yang datang akan menjalankan proses layanan

**3. Proses Layanan (`service_process`):**
- Alur: Datang → **Request** Resource → **Delay** (layanan) → **Release** Resource → Pergi
- Mencatat waktu kedatangan, waktu mulai layanan, waktu selesai
- Menghitung queue time (waktu tunggu) = waktu mulai - waktu datang

**4. Data Logging:**
- Menyimpan semua data customer ke dalam list
- Mencatat panjang antrian setiap saat
- Data disimpan dalam format yang mudah dianalisis (DataFrame)

**5. Utilization Tracking:**
- Menghitung utilization rate loket (persentase waktu loket sibuk)
- Metrik penting untuk efisiensi operasional

In [ ]:
class DriveThruSimulation:
    """Class untuk simulasi sistem antrian drive-thru"""
    
    def __init__(self, env, num_counters, warm_up_period):
        """
        Inisialisasi simulasi
        
        Parameters:
        - env: SimPy environment
        - num_counters: Jumlah loket yang tersedia
        - warm_up_period: Durasi warm-up period (menit)
        """
        self.env = env
        self.counter = simpy.Resource(env, capacity=num_counters)
        self.num_counters = num_counters
        self.warm_up_period = warm_up_period
        
        # Data logging
        self.customers_data = []
        self.queue_length = []  # (waktu, panjang_antrian)
        self.customer_count = 0
        
        # Tracking utilization
        self.busy_time = 0
        self.total_time = 0
    
    def car_arrival_generator(self):
        """
        Generator untuk membangkitkan kedatangan mobil
        Menggunakan distribusi eksponensial dengan mean = INTER_ARRIVAL_MEAN
        """
        while True:
            # Waktu antar kedatangan (Exponential distribution)
            inter_arrival = np.random.exponential(INTER_ARRIVAL_MEAN)
            yield self.env.timeout(inter_arrival)
            
            # Buat customer baru
            self.customer_count += 1
            customer_id = self.customer_count
            
            # Start proses layanan untuk customer ini
            self.env.process(self.service_process(customer_id))
    
    def service_process(self, customer_id):
        """
        Proses layanan untuk setiap mobil/customer
        Alur: Datang → Request Resource → Proses/Delay → Release Resource → Pergi
        """
        # Waktu kedatangan
        arrival_time = self.env.now
        
        # Record panjang antrian saat ini
        queue_len = len(self.counter.queue)
        self.queue_length.append((arrival_time, queue_len))
        
        # Request resource (loket)
        with self.counter.request() as request:
            yield request  # Tunggu sampai loket tersedia
            
            # Waktu mulai dilayani
            service_start_time = self.env.now
            
            # Waktu layanan (Uniform distribution)
            service_duration = np.random.uniform(SERVICE_TIME_MIN, SERVICE_TIME_MAX)
            
            # Proses layanan (delay)
            yield self.env.timeout(service_duration)
            
            # Waktu selesai dilayani
            service_end_time = self.env.now
            
            # Hitung durasi antrian (Queue Time)
            queue_time = service_start_time - arrival_time
            
            # Hitung total waktu di sistem
            total_time_in_system = service_end_time - arrival_time
            
            # Log data (hanya setelah warm-up period)
            if arrival_time >= self.warm_up_period:
                self.customers_data.append({
                    'customer_id': customer_id,
                    'arrival_time': arrival_time,
                    'service_start_time': service_start_time,
                    'service_end_time': service_end_time,
                    'queue_time': queue_time,
                    'service_duration': service_duration,
                    'total_time_in_system': total_time_in_system
                })
    
    def run_simulation(self, sim_time):
        """
        Jalankan simulasi
        """
        # Start generator kedatangan
        self.env.process(self.car_arrival_generator())
        
        # Run simulasi sampai waktu yang ditentukan
        self.env.run(until=sim_time)
        
        # Convert data ke DataFrame
        df = pd.DataFrame(self.customers_data)
        
        return df
    
    def calculate_utilization(self, df):
        """
        Hitung utilization rate dari loket
        """
        if len(df) == 0:
            return 0
        
        total_service_time = df['service_duration'].sum()
        analysis_period = ANALYSIS_TIME
        
        # Utilization = Total Service Time / (Number of Counters × Analysis Period)
        utilization = (total_service_time / (self.num_counters * analysis_period)) * 100
        
        return utilization

print("✓ Class DriveThruSimulation berhasil didefinisikan")

---
## 4. Eksekusi Skenario A: 1 Loket (Baseline)

### 🚗 Menjalankan Skenario A (Baseline: 1 Loket)

Cell ini menjalankan **Skenario A** sebagai kondisi baseline:
- Hanya **1 loket drive-thru** yang aktif
- Simulasi dijalankan selama 1100 menit (termasuk 100 menit warm-up)
- Data yang dicatat hanya setelah warm-up period untuk menghindari bias

**Proses yang terjadi:**
1. Create SimPy environment baru
2. Inisialisasi simulasi dengan capacity=1 loket
3. Jalankan simulasi hingga waktu selesai
4. Simpan hasil ke DataFrame untuk analisis

**Output yang diharapkan:**
- Total mobil yang dilayani
- Sample data 5 mobil pertama dengan kolom: customer_id, arrival_time, queue_time, service_duration, dll.

In [ ]:
print("\n" + "="*60)
print("MENJALANKAN SKENARIO A: 1 LOKET DRIVE-THRU")
print("="*60)

# Reset random seed
np.random.seed(42)

# Buat environment SimPy
env_A = simpy.Environment()

# Buat simulasi dengan 1 loket
sim_A = DriveThruSimulation(env_A, num_counters=1, warm_up_period=WARM_UP_PERIOD)

# Jalankan simulasi
df_scenario_A = sim_A.run_simulation(SIM_TIME)

print(f"\n✓ Simulasi selesai!")
print(f"  Total mobil yang dilayani (setelah warm-up): {len(df_scenario_A)}")
print(f"  Total mobil yang datang: {sim_A.customer_count}")
print("\nSample Data (5 mobil pertama):")
print(df_scenario_A.head())

---
## 5. Eksekusi Skenario B: 2 Loket (Perbaikan)

### 🚗🚗 Menjalankan Skenario B (Perbaikan: 2 Loket)

Cell ini menjalankan **Skenario B** sebagai kondisi perbaikan:
- **2 loket drive-thru** aktif bersamaan
- Simulasi dijalankan dengan parameter yang sama untuk fair comparison
- Random seed direset ke nilai yang sama agar pola kedatangan konsisten

**Perbedaan dengan Skenario A:**
- Capacity resource ditingkatkan menjadi 2 loket
- Mobil bisa dilayani oleh 2 loket secara paralel
- Expected: Waktu tunggu lebih rendah karena kapasitas lebih besar

**Fair Comparison:**
Dengan menggunakan random seed yang sama, kita memastikan:
- Pola kedatangan mobil identik di kedua skenario
- Waktu layanan yang di-generate sama
- Perbedaan performa murni dari jumlah loket, bukan karena variasi random

In [ ]:
print("\n" + "="*60)
print("MENJALANKAN SKENARIO B: 2 LOKET DRIVE-THRU")
print("="*60)

# Reset random seed untuk fair comparison
np.random.seed(42)

# Buat environment SimPy baru
env_B = simpy.Environment()

# Buat simulasi dengan 2 loket
sim_B = DriveThruSimulation(env_B, num_counters=2, warm_up_period=WARM_UP_PERIOD)

# Jalankan simulasi
df_scenario_B = sim_B.run_simulation(SIM_TIME)

print(f"\n✓ Simulasi selesai!")
print(f"  Total mobil yang dilayani (setelah warm-up): {len(df_scenario_B)}")
print(f"  Total mobil yang datang: {sim_B.customer_count}")
print("\nSample Data (5 mobil pertama):")
print(df_scenario_B.head())

---
## 6. Analisis Statistik Deskriptif

### 📊 Analisis Statistik Deskriptif Per Skenario

Cell ini membuat fungsi untuk menampilkan **statistik deskriptif lengkap** dari hasil simulasi:

**Metrik yang Dianalisis:**

1. **Waktu Tunggu (Queue Time):**
   - Mean, Median, Min, Max, Std Dev
   - Metrik paling penting untuk kepuasan pelanggan
   
2. **Total Waktu di Sistem:**
   - Kombinasi waktu tunggu + waktu layanan
   - Menunjukkan berapa lama total customer di drive-thru

3. **Waktu Layanan:**
   - Validasi bahwa distribusi uniform 3-7 menit berjalan dengan benar
   
4. **Utilization Rate:**
   - Persentase waktu loket sibuk melayani
   - Indikator efisiensi resource

5. **Performa Layanan:**
   - Persentase mobil dengan waktu tunggu > 10 menit
   - SLA (Service Level Agreement) indicator

Output ditampilkan untuk **kedua skenario** sehingga bisa dibandingkan langsung.

In [ ]:
def print_statistics(df, scenario_name, num_counters):
    """
    Cetak statistik deskriptif untuk skenario
    """
    print("\n" + "="*70)
    print(f"STATISTIK DESKRIPTIF - {scenario_name}")
    print("="*70)
    
    # Statistik Waktu Tunggu (Queue Time)
    print("\n📊 WAKTU TUNGGU DALAM ANTRIAN (Queue Time):")
    print(f"  Rata-rata (Mean)    : {df['queue_time'].mean():.2f} menit")
    print(f"  Median              : {df['queue_time'].median():.2f} menit")
    print(f"  Minimum             : {df['queue_time'].min():.2f} menit")
    print(f"  Maksimum            : {df['queue_time'].max():.2f} menit")
    print(f"  Std Deviasi         : {df['queue_time'].std():.2f} menit")
    
    # Statistik Total Waktu di Sistem
    print("\n⏱️  TOTAL WAKTU DI SISTEM:")
    print(f"  Rata-rata (Mean)    : {df['total_time_in_system'].mean():.2f} menit")
    print(f"  Median              : {df['total_time_in_system'].median():.2f} menit")
    print(f"  Minimum             : {df['total_time_in_system'].min():.2f} menit")
    print(f"  Maksimum            : {df['total_time_in_system'].max():.2f} menit")
    
    # Statistik Waktu Layanan
    print("\n🔧 WAKTU LAYANAN (Service Time):")
    print(f"  Rata-rata (Mean)    : {df['service_duration'].mean():.2f} menit")
    print(f"  Minimum             : {df['service_duration'].min():.2f} menit")
    print(f"  Maksimum            : {df['service_duration'].max():.2f} menit")
    
    # Utilization Rate
    total_service_time = df['service_duration'].sum()
    utilization = (total_service_time / (num_counters * ANALYSIS_TIME)) * 100
    
    print("\n⚙️  UTILIZATION RATE LOKET:")
    print(f"  Utilization         : {utilization:.2f}%")
    print(f"  Total Service Time  : {total_service_time:.2f} menit")
    print(f"  Available Time      : {num_counters * ANALYSIS_TIME:.2f} menit")
    
    # Persentase mobil dengan waktu tunggu > 10 menit
    long_wait = (df['queue_time'] > 10).sum()
    pct_long_wait = (long_wait / len(df)) * 100
    
    print("\n⚠️  PERFORMA LAYANAN:")
    print(f"  Mobil dengan tunggu > 10 menit: {long_wait} ({pct_long_wait:.2f}%)")
    print(f"  Total mobil dilayani          : {len(df)}")
    print("="*70)

# Cetak statistik untuk kedua skenario
print_statistics(df_scenario_A, "SKENARIO A (1 LOKET)", 1)
print_statistics(df_scenario_B, "SKENARIO B (2 LOKET)", 2)

---
## 7. Perbandingan Langsung Kedua Skenario

### 📋 Tabel Perbandingan Komprehensif

Cell ini membuat **tabel perbandingan side-by-side** antara Skenario A dan B dengan menampilkan:

**Kolom Tabel:**
1. **Metrik**: Nama metrik yang dibandingkan
2. **Skenario A (1 Loket)**: Nilai untuk baseline
3. **Skenario B (2 Loket)**: Nilai untuk kondisi perbaikan
4. **Improvement (%)**: Persentase perbaikan dari A ke B

**Metrik yang Dibandingkan:**
- Jumlah loket
- Rata-rata waktu tunggu
- Median waktu tunggu
- Maksimum waktu tunggu
- Rata-rata total waktu di sistem
- Utilization rate
- Persentase mobil dengan tunggu lama (>10 menit)

Tabel ini memberikan **overview cepat** untuk decision making: apakah worth it menambah loket kedua?

In [ ]:
# Buat tabel perbandingan
comparison = pd.DataFrame({
    'Metrik': [
        'Jumlah Loket',
        'Rata-rata Waktu Tunggu (menit)',
        'Median Waktu Tunggu (menit)',
        'Max Waktu Tunggu (menit)',
        'Rata-rata Total Waktu (menit)',
        'Utilization Rate (%)',
        'Mobil dengan Tunggu > 10 menit (%)'
    ],
    'Skenario A (1 Loket)': [
        1,
        df_scenario_A['queue_time'].mean(),
        df_scenario_A['queue_time'].median(),
        df_scenario_A['queue_time'].max(),
        df_scenario_A['total_time_in_system'].mean(),
        (df_scenario_A['service_duration'].sum() / (1 * ANALYSIS_TIME)) * 100,
        ((df_scenario_A['queue_time'] > 10).sum() / len(df_scenario_A)) * 100
    ],
    'Skenario B (2 Loket)': [
        2,
        df_scenario_B['queue_time'].mean(),
        df_scenario_B['queue_time'].median(),
        df_scenario_B['queue_time'].max(),
        df_scenario_B['total_time_in_system'].mean(),
        (df_scenario_B['service_duration'].sum() / (2 * ANALYSIS_TIME)) * 100,
        ((df_scenario_B['queue_time'] > 10).sum() / len(df_scenario_B)) * 100
    ]
})

# Hitung improvement
comparison['Improvement (%)'] = (
    (comparison['Skenario A (1 Loket)'] - comparison['Skenario B (2 Loket)']) / 
    comparison['Skenario A (1 Loket)'] * 100
)

# Format untuk metrik pertama (jumlah loket) - tidak perlu improvement
comparison.loc[0, 'Improvement (%)'] = np.nan

print("\n" + "="*80)
print("TABEL PERBANDINGAN SKENARIO A vs SKENARIO B")
print("="*80)
print(comparison.to_string(index=False))
print("="*80)
print("\n📈 Catatan: Improvement positif = perbaikan performa")

---
## 8. Visualisasi: Histogram Waktu Tunggu

### 📊 Visualisasi: Histogram Waktu Tunggu

Cell ini membuat **histogram** untuk membandingkan distribusi waktu tunggu di kedua skenario.

**Komponen Visualisasi:**
- 2 subplot side-by-side (Skenario A vs B)
- Histogram dengan 30 bins untuk detail distribusi
- Garis merah putus-putus: Mean (rata-rata)
- Garis biru titik-titik: Median (nilai tengah)

**Interpretasi yang Dapat Dilihat:**
- **Bentuk distribusi**: Apakah normal, skewed, atau multimodal?
- **Spread**: Seberapa tersebar waktu tunggu (konsistensi)?
- **Central tendency**: Di mana sebagian besar pelanggan berada?
- **Outliers**: Apakah ada pelanggan dengan tunggu sangat lama?

**Yang Diharapkan:**
- Skenario A: Distribusi lebih lebar, mean & median tinggi
- Skenario B: Distribusi lebih terkonsentrasi di waktu tunggu rendah

In [ ]:
# Buat histogram perbandingan waktu tunggu
fig, axes = plt.subplots(1, 2, figsize=(15, 5))

# Histogram Skenario A
axes[0].hist(df_scenario_A['queue_time'], bins=30, color='#FF6B6B', 
             alpha=0.7, edgecolor='black')
axes[0].axvline(df_scenario_A['queue_time'].mean(), color='red', 
                linestyle='--', linewidth=2, label=f"Mean = {df_scenario_A['queue_time'].mean():.2f} menit")
axes[0].axvline(df_scenario_A['queue_time'].median(), color='blue', 
                linestyle=':', linewidth=2, label=f"Median = {df_scenario_A['queue_time'].median():.2f} menit")
axes[0].set_xlabel('Waktu Tunggu (menit)', fontsize=12)
axes[0].set_ylabel('Frekuensi', fontsize=12)
axes[0].set_title('Skenario A: 1 Loket\nDistribusi Waktu Tunggu', fontsize=14, fontweight='bold')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Histogram Skenario B
axes[1].hist(df_scenario_B['queue_time'], bins=30, color='#4ECDC4', 
             alpha=0.7, edgecolor='black')
axes[1].axvline(df_scenario_B['queue_time'].mean(), color='red', 
                linestyle='--', linewidth=2, label=f"Mean = {df_scenario_B['queue_time'].mean():.2f} menit")
axes[1].axvline(df_scenario_B['queue_time'].median(), color='blue', 
                linestyle=':', linewidth=2, label=f"Median = {df_scenario_B['queue_time'].median():.2f} menit")
axes[1].set_xlabel('Waktu Tunggu (menit)', fontsize=12)
axes[1].set_ylabel('Frekuensi', fontsize=12)
axes[1].set_title('Skenario B: 2 Loket\nDistribusi Waktu Tunggu', fontsize=14, fontweight='bold')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("\n📊 Interpretasi Histogram:")
print("  - Skenario A (1 Loket): Distribusi lebih menyebar, banyak mobil menunggu lama")
print("  - Skenario B (2 Loket): Distribusi lebih terkonsentrasi di waktu tunggu rendah")

---
## 9. Visualisasi: Perbandingan Box Plot

### 📦 Visualisasi: Box Plot Perbandingan

Cell ini membuat **box plot** untuk membandingkan distribusi waktu tunggu dan total waktu di sistem.

**Apa itu Box Plot?**
Box plot menampilkan 5 statistik penting:
- **Q1 (Kuartil 1)**: 25% data di bawah ini
- **Median (Q2)**: 50% data di bawah ini
- **Q3 (Kuartil 3)**: 75% data di bawah ini
- **IQR (Interquartile Range)**: Q3 - Q1 (spread data tengah)
- **Outliers**: Titik-titik di luar whiskers (data ekstrem)

**2 Box Plot yang Dibuat:**
1. **Waktu Tunggu**: Perbandingan queue time
2. **Total Waktu di Sistem**: Perbandingan keseluruhan pengalaman

**Interpretasi Cepat:**
- Box lebih rendah = performa lebih baik
- Box lebih kecil = lebih konsisten
- Whisker lebih pendek = variabilitas lebih rendah
- Outliers sedikit = sistem lebih stabil

In [ ]:
# Buat data untuk box plot
df_scenario_A['Skenario'] = '1 Loket'
df_scenario_B['Skenario'] = '2 Loket'

df_combined = pd.concat([df_scenario_A, df_scenario_B])

# Buat box plot
fig, axes = plt.subplots(1, 2, figsize=(14, 6))

# Box plot waktu tunggu
sns.boxplot(x='Skenario', y='queue_time', data=df_combined, 
            palette=['#FF6B6B', '#4ECDC4'], ax=axes[0])
axes[0].set_ylabel('Waktu Tunggu (menit)', fontsize=12)
axes[0].set_xlabel('Skenario', fontsize=12)
axes[0].set_title('Perbandingan Waktu Tunggu\nKedua Skenario', fontsize=14, fontweight='bold')
axes[0].grid(True, alpha=0.3, axis='y')

# Box plot total waktu di sistem
sns.boxplot(x='Skenario', y='total_time_in_system', data=df_combined, 
            palette=['#FF6B6B', '#4ECDC4'], ax=axes[1])
axes[1].set_ylabel('Total Waktu di Sistem (menit)', fontsize=12)
axes[1].set_xlabel('Skenario', fontsize=12)
axes[1].set_title('Perbandingan Total Waktu di Sistem\nKedua Skenario', fontsize=14, fontweight='bold')
axes[1].grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.show()

print("\n📦 Interpretasi Box Plot:")
print("  - Box yang lebih rendah = performa lebih baik (waktu tunggu lebih singkat)")
print("  - Skenario B menunjukkan median dan quartile yang jauh lebih rendah")

---
## 10. Visualisasi: Line Chart Panjang Antrian

### 📈 Visualisasi: Line Chart Panjang Antrian Seiring Waktu

Cell ini membuat **line chart** yang menunjukkan evolusi panjang antrian sepanjang simulasi.

**Data yang Diplot:**
- X-axis: Waktu simulasi (menit)
- Y-axis: Jumlah mobil dalam antrian
- 2 garis: Skenario A (merah) vs Skenario B (biru)

**Insight yang Bisa Dilihat:**
1. **Dinamika antrian**: Apakah antrian stabil atau fluktuatif?
2. **Peak times**: Kapan antrian mencapai puncak?
3. **Build-up**: Apakah antrian terus bertambah (sistem overwhelmed)?
4. **Recovery**: Seberapa cepat sistem recover setelah peak?

**Yang Diharapkan:**
- Skenario A: Antrian lebih panjang dan sering build-up
- Skenario B: Antrian lebih pendek dan lebih stabil

**Catatan:** Untuk kejelasan, hanya 200 menit pertama setelah warm-up yang ditampilkan.

In [ ]:
# Ambil data panjang antrian dari kedua simulasi
queue_A = pd.DataFrame(sim_A.queue_length, columns=['time', 'queue_length'])
queue_B = pd.DataFrame(sim_B.queue_length, columns=['time', 'queue_length'])

# Filter data setelah warm-up period
queue_A = queue_A[queue_A['time'] >= WARM_UP_PERIOD]
queue_B = queue_B[queue_B['time'] >= WARM_UP_PERIOD]

# Buat line chart
plt.figure(figsize=(15, 6))

# Plot untuk rentang waktu tertentu (misalnya 200 menit pertama setelah warm-up)
plot_duration = 200
queue_A_plot = queue_A[queue_A['time'] <= WARM_UP_PERIOD + plot_duration]
queue_B_plot = queue_B[queue_B['time'] <= WARM_UP_PERIOD + plot_duration]

plt.plot(queue_A_plot['time'], queue_A_plot['queue_length'], 
         color='#FF6B6B', alpha=0.7, linewidth=1.5, label='Skenario A (1 Loket)')
plt.plot(queue_B_plot['time'], queue_B_plot['queue_length'], 
         color='#4ECDC4', alpha=0.7, linewidth=1.5, label='Skenario B (2 Loket)')

plt.xlabel('Waktu Simulasi (menit)', fontsize=12)
plt.ylabel('Panjang Antrian (jumlah mobil)', fontsize=12)
plt.title('Panjang Antrian Seiring Waktu\n(200 menit pertama setelah warm-up)', 
          fontsize=14, fontweight='bold')
plt.legend(fontsize=11)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

# Statistik panjang antrian
print("\n📊 STATISTIK PANJANG ANTRIAN:")
print("\nSkenario A (1 Loket):")
print(f"  Rata-rata panjang antrian: {queue_A['queue_length'].mean():.2f} mobil")
print(f"  Maksimum panjang antrian : {queue_A['queue_length'].max()} mobil")

print("\nSkenario B (2 Loket):")
print(f"  Rata-rata panjang antrian: {queue_B['queue_length'].mean():.2f} mobil")
print(f"  Maksimum panjang antrian : {queue_B['queue_length'].max()} mobil")

---
## 11. Kesimpulan dan Rekomendasi

### 📝 Kesimpulan dan Rekomendasi Bisnis

Cell ini menghasilkan **analisis cost-benefit komprehensif** dan rekomendasi berdasarkan hasil simulasi.

**Analisis yang Dilakukan:**

1. **Temuan Utama:**
   - Pengurangan waktu tunggu (dalam persentase)
   - Perbaikan kepuasan pelanggan (customer dengan tunggu lama)
   - Perubahan efisiensi operasional (utilization rate)

2. **Cost-Benefit Analysis:**
   - **Benefits**: Waktu tunggu turun, kepuasan naik, throughput meningkat
   - **Costs**: Investasi loket, gaji karyawan, maintenance, utilization turun

3. **Rekomendasi:**
   - Jika improvement > 50% dan long wait > 20% → SANGAT DIREKOMENDASIKAN
   - Jika improvement moderate → PERTIMBANGKAN dengan hati-hati
   - Reasoning lengkap tentang ROI

4. **Saran Tambahan:**
   - Implementasi sistem dinamis (1-2 loket tergantung jam)
   - Optimasi proses untuk mengurangi service time
   - Monitoring pola kedatangan

**Output:** Kesimpulan yang actionable untuk decision maker restoran.

In [ ]:
# Hitung improvement metrics
avg_wait_A = df_scenario_A['queue_time'].mean()
avg_wait_B = df_scenario_B['queue_time'].mean()
improvement_pct = ((avg_wait_A - avg_wait_B) / avg_wait_A) * 100

util_A = (df_scenario_A['service_duration'].sum() / (1 * ANALYSIS_TIME)) * 100
util_B = (df_scenario_B['service_duration'].sum() / (2 * ANALYSIS_TIME)) * 100

long_wait_A_pct = ((df_scenario_A['queue_time'] > 10).sum() / len(df_scenario_A)) * 100
long_wait_B_pct = ((df_scenario_B['queue_time'] > 10).sum() / len(df_scenario_B)) * 100

print("\n" + "="*80)
print("KESIMPULAN DAN REKOMENDASI")
print("="*80)

print("\n🎯 TEMUAN UTAMA:")
print(f"\n1. PENGURANGAN WAKTU TUNGGU:")
print(f"   • Skenario A (1 loket): Rata-rata tunggu {avg_wait_A:.2f} menit")
print(f"   • Skenario B (2 loket): Rata-rata tunggu {avg_wait_B:.2f} menit")
print(f"   • IMPROVEMENT: {improvement_pct:.1f}% pengurangan waktu tunggu")

print(f"\n2. KEPUASAN PELANGGAN:")
print(f"   • Skenario A: {long_wait_A_pct:.1f}% pelanggan menunggu > 10 menit")
print(f"   • Skenario B: {long_wait_B_pct:.1f}% pelanggan menunggu > 10 menit")
print(f"   • IMPROVEMENT: Pengurangan signifikan pelanggan dengan waktu tunggu lama")

print(f"\n3. EFISIENSI OPERASIONAL:")
print(f"   • Skenario A: Utilization {util_A:.1f}% (1 loket sangat sibuk)")
print(f"   • Skenario B: Utilization {util_B:.1f}% per loket (lebih seimbang)")
print(f"   • CATATAN: Utilization tinggi di Skenario A menunjukkan sistem overwhelmed")

print("\n\n💡 ANALISIS COST-BENEFIT:")
print("\n✅ KEUNTUNGAN PENAMBAHAN LOKET (Skenario B):")
print(f"   1. Waktu tunggu berkurang {improvement_pct:.1f}%")
print(f"   2. Kepuasan pelanggan meningkat drastis")
print(f"   3. Mengurangi risiko pelanggan pergi karena antrian panjang")
print(f"   4. Throughput sistem lebih stabil")
print(f"   5. Pengalaman pelanggan lebih baik (waktu tunggu lebih predictable)")

print("\n💰 BIAYA YANG PERLU DIPERTIMBANGKAN:")
print("   1. Biaya investasi loket tambahan (peralatan, infrastruktur)")
print("   2. Biaya operasional (gaji karyawan tambahan)")
print("   3. Biaya maintenance loket kedua")
print(f"   4. Trade-off: Utilization turun ke {util_B:.1f}% (ada idle time)")

print("\n\n📋 REKOMENDASI FINAL:")
if improvement_pct > 50 and long_wait_A_pct > 20:
    print("\n✅ SANGAT DIREKOMENDASIKAN untuk menambah loket ke 2!")
    print("\nAlasan:")
    print(f"   • Improvement waktu tunggu sangat signifikan ({improvement_pct:.1f}%)")
    print(f"   • Persentase pelanggan dengan tunggu lama turun drastis")
    print(f"   • ROI (Return on Investment) kemungkinan positif karena:")
    print(f"     - Peningkatan kepuasan pelanggan")
    print(f"     - Pengurangan customer churn")
    print(f"     - Kemampuan melayani lebih banyak pelanggan di jam sibuk")
else:
    print("\n⚠️ PERTIMBANGKAN DENGAN HATI-HATI")
    print("   • Improvement ada, tapi mungkin tidak sebanding dengan biaya")
    print("   • Pertimbangkan solusi alternatif (optimasi proses, dll)")

print("\n\n🔍 SARAN TAMBAHAN:")
print("   1. Implementasikan sistem loket dinamis:")
print("      - 1 loket saat sepi (pagi/sore hari)")
print("      - 2 loket saat jam sibuk (lunch time, dinner time)")
print("   2. Monitor pola kedatangan pelanggan untuk optimasi jadwal")
print("   3. Pertimbangkan sistem pre-order untuk mengurangi waktu layanan")
print("   4. Training karyawan untuk meningkatkan kecepatan layanan")

print("\n" + "="*80)
print("AKHIR ANALISIS")
print("="*80)

---
## 12. Export Data untuk Analisis Lebih Lanjut (Opsional)

### 💾 Export Data (Opsional)

Cell ini menyediakan opsi untuk **export data hasil simulasi** ke file CSV jika diperlukan analisis lebih lanjut.

**Data yang Tersedia:**
- `df_scenario_A`: Data lengkap semua customer di Skenario A
- `df_scenario_B`: Data lengkap semua customer di Skenario B  
- `df_combined`: Data gabungan kedua skenario

**Kegunaan Export:**
- Analisis statistik lanjutan di software lain (Excel, SPSS, R)
- Membuat laporan atau presentasi
- Dokumentasi hasil simulasi
- Sharing data dengan stakeholder

**Cara Menggunakan:**
- Uncomment (hapus #) baris kode untuk export
- File CSV akan tersimpan di direktori yang sama dengan notebook
- Data bisa dibuka dengan Excel atau text editor

**Note:** Export bersifat opsional dan tidak mempengaruhi analisis utama.

In [ ]:
# Export data ke CSV (jika diperlukan)
# df_scenario_A.to_csv('scenario_A_data.csv', index=False)
# df_scenario_B.to_csv('scenario_B_data.csv', index=False)

# Tampilkan ringkasan akhir
print("\n✅ SIMULASI SELESAI!")
print("\n📊 Data yang tersedia untuk analisis lebih lanjut:")
print("   • df_scenario_A: Data lengkap Skenario A (1 loket)")
print("   • df_scenario_B: Data lengkap Skenario B (2 loket)")
print("   • df_combined: Data gabungan untuk perbandingan")
print("\n💾 Untuk export data, uncomment baris kode di cell ini.")